In [1]:
import re
from urllib import request
import gzip
import shutil
import requests
import pandas as pd
import sqlalchemy
import urllib
from multiprocessing.pool import ThreadPool
import pyodbc

In [ ]:
def download_url(url):
    # Download process
    print("downloading: ",url)
    file_title = re.split(pattern='/', string=url)[-1]
    urlrtv = request.urlretrieve(url=url, filename=file_title)
    
    # for ".tsv" to ".csv"
    title = re.split(pattern=r'\.tsv', string=file_title)[0] +".csv"
    
    # Unzip ".gz" file
    with gzip.open(file_title, 'rb') as f_in:
        with open(title, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [ ]:
# URL List
urls = ["https://datasets.imdbws.com/title.episode.tsv.gz"
        ,"https://datasets.imdbws.com/title.ratings.tsv.gz"
        ,"https://datasets.imdbws.com/title.akas.tsv.gz"
        ,"https://datasets.imdbws.com/title.basics.tsv.gz"
        ,"https://datasets.imdbws.com/title.crew.tsv.gz"
        ,"https://datasets.imdbws.com/title.principals.tsv.gz"
        ,"https://datasets.imdbws.com/name.basics.tsv.gz"
        ]
# Run 5 multiple threads. Each call will take the next element in urls list
results = ThreadPool(5).imap_unordered(download_url, urls)
for r in results:
    print(r)

In [2]:
title_principals = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.principals.csv", sep= "\t", nrows=1000)
title_akas = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.akas.csv", sep= "\t", nrows=1000)
name_basics = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\name.basics.csv", sep= "\t", nrows=1000)
title_basics = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.basics.csv", sep= "\t", nrows=1000)
title_crew = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.crew.csv", sep= "\t", nrows=1000)
title_episode = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.episode.csv", sep= "\t", nrows=1000)
title_ratings = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.ratings.csv", sep= "\t", nrows=1000)

In [15]:
title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,\N,\N,original,\N,1
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita,US,\N,imdbDisplay,\N,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
4,tt0000001,5,Καρμενσίτα,GR,\N,imdbDisplay,\N,0


In [4]:
from neo4j import GraphDatabase, basic_auth
import pyneoinstance

https://neo4j.com/developer-blog/scoobygraph-2/

In [33]:
# Python file - connection to Neo4j

# Retrieving the Neo4j connection credentials from the config.yaml file
configs=pyneoinstance.load_yaml_file('config.yaml')
creds=configs['credentials']

# Establishing the Neo4j connection
neo4j=pyneoinstance.Neo4jInstance(creds['uri'],creds['user'],creds['password'])

In [20]:
# Python - loading nodes
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_akas'], data = title_akas)


e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 8000}

In [21]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_basics'],data = title_basics)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 9000}

In [22]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_crew'],data = title_crew)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 3000}

In [23]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_episode'],data = title_episode)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 4000}

In [34]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_principals'],data = title_principals)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 6000}

In [26]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_ratings'],data = title_ratings)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 3000}

In [32]:
neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_namebasics'],data = name_basics)

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 1000, 'nodes_created': 1000, 'properties_set': 6000}

In [ ]:
# Load monster, subtype, type, and species nodes/relationships
# neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_monsters'],data = title_principals[~title_principals.monster_name.isna()])

In [ ]:
# Python script: loading main characters
# Loading the main characters
neo4j.execute_write_query(configs['cypher']['load_nodes']['load_main_characters'])

In [ ]:
# Python - Creating APPEARS_WITH relationships
neo4j.execute_write_query(configs['cypher']['load_relationships']['APPEARS_WITH'])

In [37]:
driver = GraphDatabase.driver(creds['uri'], auth=basic_auth(creds['user'], creds['password']))

In [35]:
cypher_query = '''
MATCH (n)
RETURN n AS count
LIMIT $limit
'''

In [38]:

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()


C:\Users\halo4\AppData\Local\Temp\ipykernel_10848\212345830.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


{'types': 'original', 'index_': 1, 'titleID': 'tt0000001', 'isoriginaltitle': 1, 'language': '\\N', 'attributes': '\\N', 'title_name': 'Carmencita', 'region': '\\N'}
{'types': '\\N', 'index_': 2, 'titleID': 'tt0000001', 'isoriginaltitle': 0, 'language': '\\N', 'attributes': 'literal title', 'title_name': 'Carmencita', 'region': 'DE'}
{'types': 'imdbDisplay', 'index_': 3, 'titleID': 'tt0000001', 'isoriginaltitle': 0, 'language': '\\N', 'attributes': '\\N', 'title_name': 'Carmencita', 'region': 'US'}
{'types': 'imdbDisplay', 'index_': 4, 'titleID': 'tt0000001', 'isoriginaltitle': 0, 'language': '\\N', 'attributes': '\\N', 'title_name': 'Carmencita - spanyol tánc', 'region': 'HU'}
{'types': 'imdbDisplay', 'index_': 5, 'titleID': 'tt0000001', 'isoriginaltitle': 0, 'language': '\\N', 'attributes': '\\N', 'title_name': 'Καρμενσίτα', 'region': 'GR'}
{'types': 'imdbDisplay', 'index_': 6, 'titleID': 'tt0000001', 'isoriginaltitle': 0, 'language': '\\N', 'attributes': '\\N', 'title_name': 'Кармен